In [24]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
import torch

SRC_LANGUAGE = 'arabic' # persian / south azerbaijani that uses arabic script
TGT_LANGUAGE = 'latin'

# Place-holders
token_transform = {}
vocab_transform = {}

funky = lambda x : list(x.strip().lower())
token_transform[SRC_LANGUAGE] = funky
token_transform[TGT_LANGUAGE] = funky
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']


In [14]:
vocab_transform[TGT_LANGUAGE] = torch.load('vocab_'+TGT_LANGUAGE+'.pth')
vocab_transform[SRC_LANGUAGE] = torch.load('vocab_'+SRC_LANGUAGE+'.pth')


In [15]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 200):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [16]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

ar2la

In [17]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform['arabic'])
TGT_VOCAB_SIZE = len(vocab_transform['latin'])
BATCH_SIZE = 128

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

xn = "_".join([str(x) for x in [EMB_SIZE,NHEAD,FFN_HID_DIM,NUM_ENCODER_LAYERS,NUM_DECODER_LAYERS]])

xm = 'ar2la'
MPATH = 'azeri_'+xm+xn+'.pth'

# load to gpu
if torch.cuda.is_available():
    print('Loading to GPU')
    device = torch.device("cuda")
    transformer.load_state_dict(torch.load(MPATH))
    transformer.to(device)
# load to cpu
else:
    print('Loading to CPU')
    device = torch.device('cpu')
    transformer.load_state_dict(torch.load(MPATH, map_location=device))

Loading to GPU


la2ar

In [18]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform['latin'])
TGT_VOCAB_SIZE = len(vocab_transform['arabic'])
BATCH_SIZE = 128

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer2 = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

xn = "_".join([str(x) for x in [EMB_SIZE,NHEAD,FFN_HID_DIM,NUM_ENCODER_LAYERS,NUM_DECODER_LAYERS]])
# mistake in the file name
xm = 'la2ar'

MPATH = 'azeri_'+xm+xn+'.pth'

# load to gpu
if torch.cuda.is_available():
    print('Loading to GPU')
    device = torch.device("cuda")
    transformer2.load_state_dict(torch.load(MPATH))
    transformer2.to(device)
# load to cpu
else:
    print('Loading to CPU')
    device = torch.device('cpu')
    transformer2.load_state_dict(torch.load(MPATH, map_location=device))

Loading to GPU


In [19]:
from torch.nn.utils.rnn import pad_sequence

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln],
                                               vocab_transform[ln],
                                               tensor_transform)



In [20]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def transliterate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform['arabic'](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform['latin'].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

def transliterate2(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform['latin'](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform['arabic'].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [21]:
def prep(inx,mx,mo,tfx):
    vc= vocab_transform[mx]
    allowed = vc.get_itos()

    inx = inx.split(' ')
    #[]

    out = list()
    for word in inx:
        z = list()
        z.append([[],'y'])
        for character_ in word:
            if character_ in allowed:

                # last one was was also allowed
                if z[-1][1] =='y':
                    z[-1][0].append(character_)
                # last one was not allowed
                else:
                    z.append([[character_],'y'])
            else:
                # last one was also not allowed
                if z[-1][1] == 'n':
                    z[-1][0].append(character_)
                # last one was allowed
                else:
                    z.append([[character_],'n'])
        output = []
        for sub in z:
            if sub[1]=='y':
                if sub[0]:
                    output.append("".join(mo(tfx, ''.join(sub[0])).split()))
            else:
                output.append("".join(sub[0]))
        output = " ".join(output)
        out.append(output)
    v=(" ".join(out))
    v= v.replace('  ',' ')
    return v

def ar2la(inp):
    p = prep(inp,'arabic',transliterate,transformer)
    return p

def la2ar(inp):
    p = prep(inp,'latin',transliterate2,transformer2)
    return p

persian_to_latin = ar2la
latin_to_persian = la2ar


In [22]:
persian_to_latin('خبرلر وار')

'xəbərlər var'

In [23]:
latin_to_persian('türkün dili tək sevgili istəkli dil olmaz')

'تۆرکۆن دیلی تک سئوگیلی ایستکلی دیل اۏلماز'